In [1]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from Unet3D import *
from metrics3D import *
from Datagens import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_3Dunet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  1312        ['input_1[0][0]']                
                                8, 16)                                                            
                                                                                                  
 conv3d_1 (Conv3D)              (None, 128, 128, 12  6928        ['conv3d[0][0]']                 
                                8, 16)                                                        

                                                                                                  
 up_sampling3d_2 (UpSampling3D)  (None, 64, 64, 64,   0          ['conv3d_15[0][0]']              
                                64)                                                               
                                                                                                  
 conv3d_16 (Conv3D)             (None, 64, 64, 64,   16416       ['up_sampling3d_2[0][0]']        
                                32)                                                               
                                                                                                  
 concatenate_2 (Concatenate)    (None, 64, 64, 64,   0           ['conv3d_3[0][0]',               
                                64)                               'conv3d_16[0][0]']              
                                                                                                  
 conv3d_17

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [4]:
print(len(train_ids))
print(len(val_ids))
print(len(test_ids))

200
60
36


In [5]:
training_generator = DataGenerator3D(train_ids)
valid_generator = DataGenerator3D(val_ids)
test_generator = DataGenerator3D(test_ids)

In [6]:
csv_logger = CSVLogger('Logs/training/test.log', separator=',', append=True)

eval_csv_logger = CSVLogger('Logs/testing/test.log', separator=',', append=True)

callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [7]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/3D/test.h5")

Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv3d_19/Conv3D' defined at (most recent call last):
    File "C:\Users\VegardEikenes\anaconda3\envs\test2\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\VegardEikenes\anaconda3\envs\test2\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\VegardEikenes\anaconda3\envs\test2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\VegardEikenes\anaconda3\envs\test2\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\VegardEikenes\anaconda3\envs\test2\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\VegardEikenes\anaconda3\envs\test2\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever(

In [ ]:
model = keras.models.load_model('Models/3D/U_net_200_flair_t1ce_t2.h5', 
                                   custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
                                                   "dice_coef": dice_coef,
                                                   "dice_coef_necrotic": dice_coef_necrotic,
                                                   "dice_coef_edema": dice_coef_edema,
                                                   "dice_coef_enhancing": dice_coef_enhancing
                                                  }, compile=True)

In [ ]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

In [ ]:
valid_csv_logger = CSVLogger('Logs/validation/3D_U_net_t1ce_t2_t1.log', separator=',', append=True)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)